# [XXIII Simpósio em Sistemas Computacionais de Alto Desempenho](https://wscad.ufsc.br/)

## OneAPI: Uma Abordagem para a Computação Heterogênea Centrada no Desenvolvedor

### [Ricardo Menotti](mailto:menotti@ufscar.br) e [Tiago da Conceição Oliveira](mailto:tiago.conceicao@fieb.org.br )

## Laboratórios

### 1. [Filtro Sobel](lab1/lab1.ipynb)

### 2. [Bitonic Sort](lab2/lab2.ipynb)

### 3. [Transformada de Hough](lab3/lab3.ipynb)

## Referências

- https://github.com/menotti/sycl-wscad-2022
- https://oneapi.io/